<a href="https://colab.research.google.com/github/aderdouri/ql_web_app/blob/master/ql_notebooks/fdheston.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install QuantLib-Python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.3 MB/s eta 0:00:00


In [ ]:
class FdHestonTests(unittest.TestCase):

    def setUp(self):
        """
        Common setup for evaluation date if needed,
        though most tests set it locally.
        """
        self.saved_eval_date = ql.Settings.instance().evaluationDate
        # Some tests set their own eval date. This ensures it's restored.

    def tearDown(self):
        ql.Settings.instance().evaluationDate = self.saved_eval_date

    def testFdmHestonVarianceMesher(self):
        print("Testing FDM Heston variance mesher...")

        today = ql.Date(22, ql.February, 2018)
        dc = ql.Actual365Fixed()
        ql.Settings.instance().evaluationDate = today

        process = ql.HestonProcess(
            ql.YieldTermStructureHandle(ql.FlatForward(today, 0.02, dc)),
            ql.YieldTermStructureHandle(ql.FlatForward(today, 0.02, dc)),
            ql.QuoteHandle(ql.SimpleQuote(100.0)),
            0.09, 1.0, 0.09, 0.2, -0.5)

        mesher = ql.FdmHestonVarianceMesher(5, process, 1.0)
        locations = list(mesher.locations()) # Convert QL Vector to Python list

        expected_locations = [
            0.0, 6.652314e-02, 9.000000e-02, 1.095781e-01, 2.563610e-01
        ]
        tol = 1e-6
        self.assertEqual(len(locations), len(expected_locations))
        for i in range(len(locations)):
            self.assertAlmostEqual(locations[i], expected_locations[i], delta=tol,
                                   msg=(f"Failed to reproduce Heston variance mesh at index {i}\n"
                                        f"    calculated: {locations[i]:.8e}\n"
                                        f"    expected:   {expected_locations[i]:.8e}\n"
                                        f"    difference: {abs(locations[i] - expected_locations[i]):.2e}"))

        lVol = ql.LocalVolTermStructureHandle(ql.LocalConstantVol(today, 2.5, dc))

        constSlvMesher = ql.FdmHestonLocalVolatilityVarianceMesher(5, process, lVol, 1.0)

        expectedVol = 2.5 * mesher.volaEstimate()
        calculatedVol = constSlvMesher.volaEstimate()
        diff = abs(calculatedVol - expectedVol)
        self.assertLessEqual(diff, tol,
                             msg=(f"Failed to reproduce Heston local volatility variance estimate (constSlvMesher)\n"
                                  f"    calculated: {calculatedVol:.8e}\n"
                                  f"    expected:   {expectedVol:.8e}\n"
                                  f"    difference: {diff:.2e}"))

        alpha = 0.01
        leverageFct = ql.LocalVolTermStructureHandle(
            ParableLocalVolatility(today, 100.0, alpha, dc))

        slvMesher = ql.FdmHestonLocalVolatilityVarianceMesher(
            5, process, leverageFct, 0.5, 1, 0.01) # maturity, timeStepsOnAvg, timeStepsPerYear

        initialVolEstimate = ql.FdmHestonVarianceMesher(5, process, 0.5, 1, 0.01).volaEstimate()

        # Mathematica solution comment implies 0.455881 for the integral part
        leverageAvg = 0.455881 / (1.0 - 0.02) # Integral part / (1 - prob of hitting bounds)

        volaEstExpected = 0.5 * (leverageAvg + leverageFct.localVol(0.0, 100.0)) * initialVolEstimate

        volaEstCalculated = slvMesher.volaEstimate()

        # C++ test uses 0.001 for this specific tolerance
        self.assertAlmostEqual(volaEstCalculated, volaEstExpected, delta=0.001,
                               msg=(f"Failed to reproduce Heston local volatility variance estimate (slvMesher)\n"
                                    f"    calculated: {volaEstCalculated:.8e}\n"
                                    f"    expected:   {volaEstExpected:.8e}\n"
                                    f"    difference: {abs(volaEstCalculated - volaEstExpected):.2e}"))


    def testFdmHestonBarrierVsBlackScholes(self):
        # This test has `@precondition(if_speed(Fast))`
        # We will run it by default in Python
        print("Testing FDM with barrier option in Heston model (vs BS)...")

        # Data from "Option pricing formulas", E.G. Haug, McGraw-Hill 1998 pag. 72
        # barrierType, barrier, rebate, type, strike, s, q, r, t, v
        barrier_option_data = [
            {'bt': ql.Barrier.DownOut, 'b': 95.0, 'reb': 3.0, 'type': ql.Option.Call, 'k': 90,  's': 100.0, 'q': 0.04, 'r': 0.08, 't': 0.50, 'v': 0.25},
            {'bt': ql.Barrier.DownOut, 'b': 95.0, 'reb': 3.0, 'type': ql.Option.Call, 'k': 100, 's': 100.0, 'q': 0.00, 'r': 0.08, 't': 1.00, 'v': 0.30},
            # ... (include all 63 entries from the C++ code)
            # For brevity, I'll include a few. The full list is very long.
            {'bt': ql.Barrier.DownOut, 'b': 95.0, 'reb': 3.0, 'type': ql.Option.Call, 'k': 110, 's': 100.0, 'q': 0.04, 'r': 0.08, 't': 0.50, 'v': 0.25},
            {'bt': ql.Barrier.UpIn,   'b': 105.0,'reb': 3.0, 'type': ql.Option.Put,  'k': 110, 's': 100.0, 'q': 0.04, 'r': 0.08, 't': 0.50, 'v': 0.30}
        ]
        # NOTE: The user should paste the full list of 63 dictionaries here for a complete test.
        # As a placeholder, let's use a very small subset for the script to run.
        # This is the original list of values
        _values_cpp_struct_like = [
            # barrierType, barrier, rebate,         type, strike,     s,    q,    r,    t,    v
            ( ql.Barrier.DownOut,    95.0,    3.0, ql.Option.Call,     90, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownOut,    95.0,    3.0, ql.Option.Call,    100, 100.0, 0.00, 0.08, 1.00, 0.30),
            ( ql.Barrier.DownOut,    95.0,    3.0, ql.Option.Call,    110, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownOut,   100.0,    3.0, ql.Option.Call,     90, 100.0, 0.00, 0.08, 0.25, 0.25),
            ( ql.Barrier.DownOut,   100.0,    3.0, ql.Option.Call,    100, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownOut,   100.0,    3.0, ql.Option.Call,    110, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpOut,     105.0,    3.0, ql.Option.Call,     90, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpOut,     105.0,    3.0, ql.Option.Call,    100, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpOut,     105.0,    3.0, ql.Option.Call,    110, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,     95.0,    3.0, ql.Option.Call,    90, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,     95.0,    3.0, ql.Option.Call,   100, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,     95.0,    3.0, ql.Option.Call,   110, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,    100.0,    3.0, ql.Option.Call,    90, 100.0, 0.00, 0.08, 0.25, 0.25),
            ( ql.Barrier.DownIn,    100.0,    3.0, ql.Option.Call,   100, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,    100.0,    3.0, ql.Option.Call,   110, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpIn,      105.0,    3.0, ql.Option.Call,    90, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpIn,      105.0,    3.0, ql.Option.Call,   100, 100.0, 0.00, 0.08, 0.40, 0.25),
            ( ql.Barrier.UpIn,      105.0,    3.0, ql.Option.Call,   110, 100.0, 0.04, 0.08, 0.50, 0.15),
            ( ql.Barrier.DownOut,    95.0,    3.0, ql.Option.Call,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownOut,    95.0,    3.0, ql.Option.Call,   100, 100.0, 0.00, 0.08, 0.40, 0.35),
            ( ql.Barrier.DownOut,    95.0,    3.0, ql.Option.Call,   110, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownOut,   100.0,    3.0, ql.Option.Call,    90, 100.0, 0.04, 0.08, 0.50, 0.15),
            ( ql.Barrier.DownOut,   100.0,    3.0, ql.Option.Call,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownOut,   100.0,    3.0, ql.Option.Call,   110, 100.0, 0.00, 0.00, 1.00, 0.20),
            ( ql.Barrier.UpOut,     105.0,    3.0, ql.Option.Call,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.UpOut,     105.0,    3.0, ql.Option.Call,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.UpOut,     105.0,    3.0, ql.Option.Call,   110, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,     95.0,    3.0, ql.Option.Call,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,     95.0,    3.0, ql.Option.Call,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,     95.0,    3.0, ql.Option.Call,   110, 100.0, 0.00, 0.08, 1.00, 0.30),
            ( ql.Barrier.DownIn,    100.0,    3.0, ql.Option.Call,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,    100.0,    3.0, ql.Option.Call,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,    100.0,    3.0, ql.Option.Call,   110, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.UpIn,      105.0,    3.0, ql.Option.Call,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.UpIn,      105.0,    3.0, ql.Option.Call,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.UpIn,      105.0,    3.0, ql.Option.Call,   110, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownOut,    95.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownOut,    95.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownOut,    95.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownOut,   100.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownOut,   100.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownOut,   100.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpOut,     105.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpOut,     105.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpOut,     105.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,     95.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,     95.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,     95.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,    100.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,    100.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.DownIn,    100.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpIn,      105.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpIn,      105.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.25),
            ( ql.Barrier.UpIn,      105.0,    3.0,  ql.Option.Put,   110, 100.0, 0.00, 0.04, 1.00, 0.15),
            ( ql.Barrier.DownOut,    95.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownOut,    95.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownOut,    95.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownOut,   100.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownOut,   100.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownOut,   100.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.UpOut,     105.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.UpOut,     105.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.UpOut,     105.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,     95.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,     95.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,     95.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,    100.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,    100.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.DownIn,    100.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 1.00, 0.15),
            ( ql.Barrier.UpIn,      105.0,    3.0,  ql.Option.Put,    90, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.UpIn,      105.0,    3.0,  ql.Option.Put,   100, 100.0, 0.04, 0.08, 0.50, 0.30),
            ( ql.Barrier.UpIn,      105.0,    3.0,  ql.Option.Put,   110, 100.0, 0.04, 0.08, 0.50, 0.30)
        ]
        keys = ['bt', 'b', 'reb', 'type', 'k', 's', 'q', 'r', 't', 'v']
        barrier_option_data = [dict(zip(keys, row)) for row in _values_cpp_struct_like]


        dc = ql.Actual365Fixed()
        todaysDate = ql.Date(28, ql.March, 2004)
        ql.Settings.instance().evaluationDate = todaysDate

        spot_quote = ql.SimpleQuote(0.0)
        q_rate_quote = ql.SimpleQuote(0.0)
        r_rate_quote = ql.SimpleQuote(0.0)
        vol_quote = ql.SimpleQuote(0.0)

        spot_h = ql.QuoteHandle(spot_quote)
        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(todaysDate, q_rate_quote, dc))
        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(todaysDate, r_rate_quote, dc))
        vol_ts_h = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(todaysDate, ql.NullCalendar(), vol_quote, dc))

        bs_process = ql.BlackScholesMertonProcess(spot_h, q_ts_h, r_ts_h, vol_ts_h)
        analytic_engine = ql.AnalyticBarrierEngine(bs_process)

        for i, P_test_case in enumerate(barrier_option_data):
            exDate = todaysDate + time_to_period_in_days(P_test_case['t'])
            exercise = ql.EuropeanExercise(exDate)

            spot_quote.setValue(P_test_case['s'])
            q_rate_quote.setValue(P_test_case['q'])
            r_rate_quote.setValue(P_test_case['r'])
            vol_quote.setValue(P_test_case['v'])

            payoff = ql.PlainVanillaPayoff(P_test_case['type'], P_test_case['k'])
            barrierOption = ql.BarrierOption(
                P_test_case['bt'], P_test_case['b'], P_test_case['reb'], payoff, exercise)

            v0 = vol_quote.value()**2
            hestonProcess = ql.HestonProcess(
                r_ts_h, q_ts_h, spot_h, v0, 1.0, v0, 0.005, 0.0) # kappa, theta, sigma_v, rho

            # Parameters from C++: tGrid=200, xGrid=101, vGrid=3
            fd_engine = ql.FdHestonBarrierEngine(
                ql.HestonModel(hestonProcess), 200, 101, 3) # tGrid, xGrid, vGrid (variance grid)

            barrierOption.setPricingEngine(fd_engine)
            calculatedHE = barrierOption.NPV()

            barrierOption.setPricingEngine(analytic_engine)
            expected = barrierOption.NPV()

            tol = 0.0025 # Relative tolerance
            if expected == 0: # Avoid division by zero if expected is 0
                abs_diff = abs(calculatedHE - expected)
                self.assertLessEqual(abs_diff, tol, # Use tol as absolute if expected is 0
                                   msg=(f"Test case {i}: Failed Heston vs BS (expected 0)\n"
                                        f"    calculated: {calculatedHE:.5f}\n"
                                        f"    expected:   {expected:.5f}\n"
                                        f"    abs_diff:    {abs_diff:.3e}"))
            else:
                rel_diff = abs(calculatedHE - expected) / expected
                self.assertLessEqual(rel_diff, tol,
                                   msg=(f"Test case {i}: Failed Heston vs BS\n"
                                        f"    calculated: {calculatedHE:.5f}\n"
                                        f"    expected:   {expected:.5f}\n"
                                        f"    rel_diff:    {rel_diff:.3e}"))

    def testFdmHestonBarrier(self):
        print("Testing FDM with barrier option for Heston model...")

        s0_quote = ql.SimpleQuote(100.0)
        s0_h = ql.QuoteHandle(s0_quote)
        dc = ql.Actual365Fixed()

        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), 0.05, dc)) # Date doesn't matter for FlatForward if eval date set
        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), 0.0, dc))

        # v0, kappa, theta, sigma_v, rho
        heston_process = ql.HestonProcess(r_ts_h, q_ts_h, s0_h, 0.04, 2.5, 0.04, 0.66, -0.8)

        evalDate = ql.Date(28, ql.March, 2004)
        ql.Settings.instance().evaluationDate = evalDate
        r_ts_h.linkTo(ql.FlatForward(evalDate, 0.05, dc)) # Re-link with correct evalDate
        q_ts_h.linkTo(ql.FlatForward(evalDate, 0.0, dc))

        exerciseDate = ql.Date(28, ql.March, 2005)
        exercise = ql.EuropeanExercise(exerciseDate)
        payoff = ql.PlainVanillaPayoff(ql.Option.Call, 100.0)

        barrierOption = ql.BarrierOption(ql.Barrier.UpOut, 135.0, 0.0, payoff, exercise)

        # tGrid=50, xGrid=400, vGrid=100
        engine = ql.FdHestonBarrierEngine(ql.HestonModel(heston_process), 50, 400, 100)
        barrierOption.setPricingEngine(engine)

        tol = 0.01
        npvExpected = 9.1530
        deltaExpected = 0.5218
        gammaExpected = -0.0354

        self.assertAlmostEqual(barrierOption.NPV(), npvExpected, delta=tol,
                               msg=f"NPV mismatch: calc={barrierOption.NPV()}, exp={npvExpected}")
        self.assertAlmostEqual(barrierOption.delta(), deltaExpected, delta=tol,
                               msg=f"Delta mismatch: calc={barrierOption.delta()}, exp={deltaExpected}")
        self.assertAlmostEqual(barrierOption.gamma(), gammaExpected, delta=tol,
                               msg=f"Gamma mismatch: calc={barrierOption.gamma()}, exp={gammaExpected}")

    def testFdmHestonAmerican(self):
        print("Testing FDM with American option in Heston model...")
        s0_quote = ql.SimpleQuote(100.0)
        s0_h = ql.QuoteHandle(s0_quote)
        dc = ql.Actual365Fixed()

        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), 0.05, dc))
        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.TARGET(), 0.0, dc))

        heston_process = ql.HestonProcess(r_ts_h, q_ts_h, s0_h, 0.04, 2.5, 0.04, 0.66, -0.8)

        evalDate = ql.Date(28, ql.March, 2004)
        ql.Settings.instance().evaluationDate = evalDate
        r_ts_h.linkTo(ql.FlatForward(evalDate, 0.05, dc))
        q_ts_h.linkTo(ql.FlatForward(evalDate, 0.0, dc))

        exerciseDate = ql.Date(28, ql.March, 2005)
        exercise = ql.AmericanExercise(evalDate, exerciseDate) # American needs start and end
        payoff = ql.PlainVanillaPayoff(ql.Option.Put, 100.0)
        option = ql.VanillaOption(payoff, exercise)

        # tGrid=200, xGrid=100, vGrid=50
        engine = ql.FdHestonVanillaEngine(ql.HestonModel(heston_process), 200, 100, 50)
        option.setPricingEngine(engine)

        tol = 0.01
        npvExpected = 5.66032
        deltaExpected = -0.30065
        gammaExpected = 0.02202

        self.assertAlmostEqual(option.NPV(), npvExpected, delta=tol,
                               msg=f"NPV mismatch: calc={option.NPV()}, exp={npvExpected}")
        self.assertAlmostEqual(option.delta(), deltaExpected, delta=tol,
                               msg=f"Delta mismatch: calc={option.delta()}, exp={deltaExpected}")
        self.assertAlmostEqual(option.gamma(), gammaExpected, delta=tol,
                               msg=f"Gamma mismatch: calc={option.gamma()}, exp={gammaExpected}")

    def testFdmHestonIkonenToivanen(self):
        print("Testing FDM Heston for Ikonen and Toivanen tests...")
        dc = ql.Actual360()
        evalDate = ql.Date(28, ql.March, 2004)
        ql.Settings.instance().evaluationDate = evalDate

        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(evalDate, 0.10, dc))
        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(evalDate, 0.0, dc))

        exerciseDate = ql.Date(26, ql.June, 2004)
        exercise = ql.AmericanExercise(evalDate, exerciseDate)
        payoff = ql.PlainVanillaPayoff(ql.Option.Put, 10.0) # Strike is fixed
        option = ql.VanillaOption(payoff, exercise)

        # Spot values are treated as strikes in the paper's table for ATM options
        # The C++ test sets s0 = strike[i]
        spot_values = [8.0, 9.0, 10.0, 11.0, 12.0]
        expected_npvs = [2.00000, 1.10763, 0.520038, 0.213681, 0.082046]
        tol = 0.001

        s0_quote = ql.SimpleQuote(0.0) # Will be updated in loop
        s0_h = ql.QuoteHandle(s0_quote)

        for i in range(len(spot_values)):
            s0_quote.setValue(spot_values[i])
            # v0, kappa, theta, sigma_v, rho
            heston_process = ql.HestonProcess(r_ts_h, q_ts_h, s0_h, 0.0625, 5.0, 0.16, 0.9, 0.1)

            # C++ FdHestonVanillaEngine(model, 100, 400) -> tGrid=100, xGrid=400, vGrid=default(50)
            engine = ql.FdHestonVanillaEngine(ql.HestonModel(heston_process), 100, 400)
            option.setPricingEngine(engine)
            calculated = option.NPV()
            self.assertAlmostEqual(calculated, expected_npvs[i], delta=tol,
                                   msg=(f"Strike: {spot_values[i]}\n"
                                        f"    calculated: {calculated:.6f}\n"
                                        f"    expected:   {expected_npvs[i]:.6f}"))

    def testFdmHestonBlackScholes(self):
        print("Testing FDM Heston with Black Scholes model...")
        dc = ql.Actual360()
        evalDate = ql.Date(28, ql.March, 2004)
        ql.Settings.instance().evaluationDate = evalDate

        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(evalDate, 0.10, dc))
        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(evalDate, 0.0, dc))
        vol_ts_h = ql.BlackVolTermStructureHandle(
            ql.BlackConstantVol(evalDate, ql.NullCalendar(), 0.25, dc))

        exerciseDate = ql.Date(26, ql.June, 2004)
        exercise = ql.EuropeanExercise(exerciseDate)
        payoff = ql.PlainVanillaPayoff(ql.Option.Put, 10.0) # Fixed strike
        option = ql.VanillaOption(payoff, exercise)

        spot_values = [8.0, 9.0, 10.0, 11.0, 12.0]
        tol = 0.0001
        s0_quote = ql.SimpleQuote(0.0)
        s0_h = ql.QuoteHandle(s0_quote)

        for spot_val in spot_values:
            s0_quote.setValue(spot_val)
            bs_process = ql.GeneralizedBlackScholesProcess(s0_h, q_ts_h, r_ts_h, vol_ts_h)
            option.setPricingEngine(ql.AnalyticEuropeanEngine(bs_process))
            expected = option.NPV()

            # Heston parameters to mimic BS: v0=vol^2, kappa=1 (or any >0), theta=v0, sigma_v=0 (or very small), rho=0
            v0_bs = 0.25**2
            heston_process_bs = ql.HestonProcess(r_ts_h, q_ts_h, s0_h, v0_bs, 1.0, v0_bs, 0.0001, 0.0)
            heston_model_bs = ql.HestonModel(heston_process_bs)

            # Hundsdorfer scheme (default if not specified, but C++ passes vGrid=3)
            # C++: FdHestonVanillaEngine(model, 100, 400, 3) -> tGrid=100, xGrid=400, vGrid=3
            engine_hund = ql.FdHestonVanillaEngine(heston_model_bs, 100, 400, 3)
            option.setPricingEngine(engine_hund)
            calculated_hund = option.NPV()
            self.assertAlmostEqual(calculated_hund, expected, delta=tol,
                                   msg=(f"Hundsdorfer: Spot {spot_val}\n"
                                        f"    calculated: {calculated_hund:.6f}\n"
                                        f"    expected:   {expected:.6f}"))

            # Explicit scheme
            # C++: FdHestonVanillaEngine(model, 4000, 400, 3, 0, FdmSchemeDesc::ExplicitEuler())
            #      tGrid=4000, xGrid=400, vGrid=3, dampingSteps=0, scheme=ExplicitEuler
            engine_expl = ql.FdHestonVanillaEngine(
                heston_model_bs, 4000, 400, 3, 0, ql.FdmSchemeDesc.ExplicitEuler())
            option.setPricingEngine(engine_expl)
            calculated_expl = option.NPV()
            self.assertAlmostEqual(calculated_expl, expected, delta=tol,
                                   msg=(f"Explicit Euler: Spot {spot_val}\n"
                                        f"    calculated: {calculated_expl:.6f}\n"
                                        f"    expected:   {expected:.6f}"))

    def testFdmHestonEuropeanWithDividends(self):
        print("Testing FDM with European option with dividends in Heston model...")
        s0_quote = ql.SimpleQuote(100.0)
        s0_h = ql.QuoteHandle(s0_quote)
        dc = ql.Actual365Fixed()
        evalDate = ql.Date(28, ql.March, 2004)
        ql.Settings.instance().evaluationDate = evalDate

        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(evalDate, 0.05, dc))
        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(evalDate, 0.0, dc)) # Initial q is 0

        heston_process = ql.HestonProcess(r_ts_h, q_ts_h, s0_h, 0.04, 2.5, 0.04, 0.66, -0.8)
        heston_model = ql.HestonModel(heston_process)

        exerciseDate = ql.Date(28, ql.March, 2005)
        # C++ uses AmericanExercise here, "EuropeanWithDividends" might be a misnomer or implies it should behave like Euro
        # Let's stick to AmericanExercise as in C++
        exercise = ql.AmericanExercise(evalDate, exerciseDate)
        payoff = ql.PlainVanillaPayoff(ql.Option.Put, 100.0)

        dividends = [5.0]
        dividend_dates = [ql.Date(28, ql.September, 2004)]

        option = ql.VanillaOption(payoff, exercise)

        # FdHestonVanillaEngine with dividends constructor:
        # model, dividendDates, dividends, tGrid, xGrid, vGrid, dampingSteps, schemeDesc
        # C++: FdHestonVanillaEngine(model, DividendVector(divDates, divs), 50, 100, 50)
        #      tGrid=50, xGrid=100, vGrid=50
        engine = ql.FdHestonVanillaEngine(
            heston_model, dividend_dates, dividends, 50, 100, 50)
        option.setPricingEngine(engine)

        tol = 0.01
        gammaTol = 0.001
        npvExpected = 7.38216
        deltaExpected = -0.397902
        gammaExpected = 0.027747

        self.assertAlmostEqual(option.NPV(), npvExpected, delta=tol, msg="NPV mismatch")
        self.assertAlmostEqual(option.delta(), deltaExpected, delta=tol, msg="Delta mismatch")
        self.assertAlmostEqual(option.gamma(), gammaExpected, delta=gammaTol, msg="Gamma mismatch")

    def testFdmHestonConvergence(self):
        # This test has `@precondition(if_speed(Fast))`
        print("Testing FDM Heston convergence...")

        # HestonTestData struct in C++
        # kappa, theta, sigma, rho, r, q, T, K
        heston_test_data = [
            {'kappa': 1.5, 'theta': 0.04, 'sigma': 0.3, 'rho': -0.9, 'r': 0.025, 'q': 0.0, 'T': 1.0, 'K': 100.0},
            {'kappa': 3.0, 'theta': 0.12, 'sigma': 0.04, 'rho': 0.6, 'r': 0.01, 'q': 0.04, 'T': 1.0, 'K': 100.0},
            {'kappa': 0.6067, 'theta': 0.0707, 'sigma': 0.2928, 'rho': -0.7571, 'r': 0.03, 'q': 0.0, 'T': 3.0, 'K': 100.0},
            {'kappa': 2.5, 'theta': 0.06, 'sigma': 0.5, 'rho': -0.1, 'r': 0.0507, 'q': 0.0469, 'T': 0.25, 'K': 100.0}
        ]

        schemes = [
            ql.FdmSchemeDesc.Hundsdorfer(),
            ql.FdmSchemeDesc.ModifiedCraigSneyd(),
            ql.FdmSchemeDesc.ModifiedHundsdorfer(),
            ql.FdmSchemeDesc.CraigSneyd(),
            ql.FdmSchemeDesc.TrBDF2(),
            ql.FdmSchemeDesc.CrankNicolson(),
        ]

        tn = [60] # time steps for FDM
        v0_list = [0.04] # initial variance list

        todaysDate = ql.Date(28, ql.March, 2004)
        ql.Settings.instance().evaluationDate = todaysDate
        dc = ql.Actual365Fixed()

        s0_quote = ql.SimpleQuote(75.0) # Fixed spot for these tests
        s0_h = ql.QuoteHandle(s0_quote)

        for i_scheme, scheme_desc in enumerate(schemes):
            for i_case, case_data in enumerate(heston_test_data):
                for t_steps in tn:
                    for v0_val in v0_list:
                        r_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(todaysDate, case_data['r'], dc))
                        q_ts_h = ql.YieldTermStructureHandle(ql.FlatForward(todaysDate, case_data['q'], dc))

                        heston_process = ql.HestonProcess(
                            r_ts_h, q_ts_h, s0_h, v0_val,
                            case_data['kappa'], case_data['theta'],
                            case_data['sigma'], case_data['rho'])
                        heston_model = ql.HestonModel(heston_process)

                        exerciseDate = todaysDate + time_to_period_in_days(case_data['T'])
                        exercise = ql.EuropeanExercise(exerciseDate)
                        payoff = ql.PlainVanillaPayoff(ql.Option.Call, case_data['K'])
                        option = ql.VanillaOption(payoff, exercise)

                        # C++: FdHestonVanillaEngine(model, j, 101, 51, 0, scheme)
                        #      tGrid=j (t_steps), xGrid=101, vGrid=51, damping=0, scheme
                        fd_engine = ql.FdHestonVanillaEngine(
                            heston_model, t_steps, 101, 51, 0, scheme_desc)
                        option.setPricingEngine(fd_engine)
                        calculated = option.NPV()

                        analytic_engine = ql.AnalyticHestonEngine(heston_model, 144) # 144 integration points
                        option.setPricingEngine(analytic_engine)
                        expected = option.NPV()

                        # C++ tolerance: std::fabs(expected - calculated)/expected > 0.02 && std::fabs(expected - calculated) > 0.002
                        rel_diff = abs(expected - calculated) / expected if expected != 0 else float('inf')
                        abs_diff = abs(expected - calculated)

                        condition = (rel_diff > 0.02 and abs_diff > 0.002)
                        self.assertFalse(condition,
                                         msg=(f"Convergence test failed: Scheme {i_scheme}, Case {i_case}, t_steps {t_steps}, v0 {v0_val}\n"
                                              f"    calculated: {calculated:.6f}\n"
                                              f"    expected:   {expected:.6f}\n"
                                              f"    rel_diff:   {rel_diff:.4f}\n"
                                              f"    abs_diff:   {abs_diff:.4f}"))

    @unittest.skipIf(not hasattr(ql.Date, 'isToday') or ql.Date(1,1,1901,1,1,1).serialNumber() == ql.Date(1,1,1901).serialNumber(),
                     "Skipping intraday pricing test: High resolution date/time might not be fully supported or enabled.")
    def testFdmHestonIntradayPricing(self):
        # This test is guarded by #ifdef QL_HIGH_RESOLUTION_DATE in C++
        # We try to check if ql.Date supports H:M:S. A more robust check might be needed.
        print("Testing FDM Heston intraday pricing...")

        option_type = ql.Option.Put
        underlying = 36.0
        strike = underlying
        dividendYield = 0.00
        riskFreeRate = 0.06
        v0 = 0.2
        kappa = 1.0
        theta = v0
        sigma = 0.0065
        rho = -0.75
        dayCounter = ql.Actual365Fixed()

        # Maturity: Date(17, May, 2014, 17, 30, 0)
        maturity_date = ql.Date(17, ql.May, 2014, 17, 30, 0)

        europeanExercise = ql.EuropeanExercise(maturity_date)
        payoff = ql.PlainVanillaPayoff(option_type, strike)
        option = ql.VanillaOption(payoff, europeanExercise)

        s0_h = ql.QuoteHandle(ql.SimpleQuote(underlying))

        # Relinkable handles
        flatTermStructure_h = ql.RelinkableYieldTermStructureHandle()
        flatDividendTS_h = ql.RelinkableYieldTermStructureHandle()

        process = ql.HestonProcess(
            flatTermStructure_h, flatDividendTS_h, s0_h,
            v0, kappa, theta, sigma, rho)
        model = ql.HestonModel(process)

        # C++: FdHestonVanillaEngine(model, 20, 100, 26, 0)
        #      tGrid=20, xGrid=100, vGrid=26, dampingSteps=0
        fdm_engine = ql.FdHestonVanillaEngine(model, 20, 100, 26, 0)
        option.setPricingEngine(fdm_engine)

        gammaExpected = [
            1.46757, 1.54696, 1.6408, 1.75409, 1.89464,
            2.07548, 2.32046, 2.67944, 3.28164, 4.64096
        ]

        for i in range(10):
            # now: Date(17, May, 2014, 15, i*15, 0)
            now_date = ql.Date(17, ql.May, 2014, 15, i * 15, 0)
            ql.Settings.instance().evaluationDate = now_date

            flatTermStructure_h.linkTo(ql.FlatForward(now_date, riskFreeRate, dayCounter))
            flatDividendTS_h.linkTo(ql.FlatForward(now_date, dividendYield, dayCounter))

            # Force recalculation (important if handles are updated)
            # option.recalculate() # May not be needed if engine pulls new dates.
            # Usually, just getting NPV() or gamma() triggers recalc if needed.

            gammaCalculated = option.gamma()
            self.assertAlmostEqual(gammaCalculated, gammaExpected[i], delta=1e-4,
                                   msg=(f"Intraday gamma mismatch at i={i} (time {now_date})\n"
                                        f"    expected:   {gammaExpected[i]:.5f}\n"
                                        f"    calculated: {gammaCalculated:.5f}"))

    def testMethodOfLinesAndCN(self):
        print("Testing method of lines to solve Heston PDEs...")
        dc = ql.Actual365Fixed()
        today = ql.Date(21, ql.February, 2018)
        ql.Settings.instance().evaluationDate = today

        spot_h = ql.QuoteHandle(ql.SimpleQuote(100.0))
        qTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.0, dc))
        rTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.0, dc))

        v0 = 0.09; kappa = 1.0; theta = v0; sigma = 0.4; rho = -0.75
        maturity_date = today + ql.Period(3, ql.Months)

        model = ql.HestonModel(
            ql.HestonProcess(rTS_h, qTS_h, spot_h, v0, kappa, theta, sigma, rho))

        xGrid = 21; vGrid = 7; tGrid = 10 # C++: (model, 10, xGrid, vGrid, 0)

        # Default scheme is Hundsdorfer
        fdmDefault = ql.FdHestonVanillaEngine(model, tGrid, xGrid, vGrid, 0)
        fdmMol = ql.FdHestonVanillaEngine(
            model, tGrid, xGrid, vGrid, 0, ql.FdmSchemeDesc.MethodOfLines())

        payoff = ql.PlainVanillaPayoff(ql.Option.Put, spot_h.value())
        # American Exercise: evalDate to maturity_date
        american_exercise = ql.AmericanExercise(today, maturity_date)
        option = ql.VanillaOption(payoff, american_exercise)

        option.setPricingEngine(fdmMol)
        calculatedMoL = option.NPV()
        option.setPricingEngine(fdmDefault)
        expected = option.NPV()

        tol = 0.005
        self.assertAlmostEqual(calculatedMoL, expected, delta=tol,
                               msg=(f"American option MoL vs Default failed\n"
                                    f"    MoL: {calculatedMoL:.5f}, Default: {expected:.5f}"))

        fdmCN = ql.FdHestonVanillaEngine(
            model, tGrid, xGrid, vGrid, 0, ql.FdmSchemeDesc.CrankNicolson())
        option.setPricingEngine(fdmCN)
        calculatedCN = option.NPV()
        self.assertAlmostEqual(calculatedCN, expected, delta=tol,
                               msg=(f"American option CN vs Default failed\n"
                                    f"    CN: {calculatedCN:.5f}, Default: {expected:.5f}"))

        # Barrier Option part
        european_exercise = ql.EuropeanExercise(maturity_date)
        barrierOption = ql.BarrierOption(
            ql.Barrier.DownOut, 85.0, 10.0, payoff, european_exercise)

        # C++: FdHestonBarrierEngine(model, 100, 31, 11) -> tGrid, xGrid, vGrid
        barrier_tGrid = 100; barrier_xGrid = 31; barrier_vGrid = 11

        barrierEngineDefault = ql.FdHestonBarrierEngine(
            model, barrier_tGrid, barrier_xGrid, barrier_vGrid)
        barrierOption.setPricingEngine(barrierEngineDefault)
        expectedBarrier = barrierOption.NPV()

        barrierEngineMoL = ql.FdHestonBarrierEngine(
            model, barrier_tGrid, barrier_xGrid, barrier_vGrid, 0, ql.FdmSchemeDesc.MethodOfLines())
        barrierOption.setPricingEngine(barrierEngineMoL)
        calculatedBarrierMoL = barrierOption.NPV()

        barrierTol = 0.01
        self.assertAlmostEqual(calculatedBarrierMoL, expectedBarrier, delta=barrierTol,
                               msg=(f"Barrier option MoL vs Default failed\n"
                                    f"    MoL: {calculatedBarrierMoL:.5f}, Default: {expectedBarrier:.5f}"))

        barrierEngineCN = ql.FdHestonBarrierEngine(
            model, barrier_tGrid, barrier_xGrid, barrier_vGrid, 0, ql.FdmSchemeDesc.CrankNicolson())
        barrierOption.setPricingEngine(barrierEngineCN)
        calculatedBarrierCN = barrierOption.NPV()
        self.assertAlmostEqual(calculatedBarrierCN, expectedBarrier, delta=barrierTol,
                               msg=(f"Barrier option CN vs Default failed\n"
                                    f"    CN: {calculatedBarrierCN:.5f}, Default: {expectedBarrier:.5f}"))

    def testSpuriousOscillations(self):
        print("Testing for spurious oscillations when solving the Heston PDEs...")
        dc = ql.Actual365Fixed()
        today = ql.Date(7, ql.June, 2018)
        ql.Settings.instance().evaluationDate = today

        spot_h = ql.QuoteHandle(ql.SimpleQuote(100.0))
        qTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.1, dc))
        rTS_h = ql.YieldTermStructureHandle(ql.FlatForward(today, 0.0, dc))

        v0 = 0.005; kappa = 1.0; theta = 0.005; sigma = 0.4; rho = -0.75
        maturity_date = today + ql.Period(1, ql.Years)

        process = ql.HestonProcess(rTS_h, qTS_h, spot_h, v0, kappa, theta, sigma, rho)
        model = ql.HestonModel(process)
        process_h = ql.HestonProcessHandle(process) # Handle needed for solver

        # Engine to get solver_desc from (tGrid=6, xGrid=200, vGrid=13)
        hestonEngine = ql.FdHestonVanillaEngine(
            model, 6, 200, 13, 0, ql.FdmSchemeDesc.TrBDF2())

        # The C++ getSolverDesc(1.0) is available in Python
        # The leverageFct argument defaults to 1.0 if not given or if handle is empty.
        # If FdHestonVanillaEngine was constructed with a leverageFct, that would be used by getSolverDesc.
        # Here, it's default (no SLV).
        solver_desc = hestonEngine.getSolverDesc()

        # Tuples: (FdmSchemeDesc, scheme_name_str, expected_spurious_bool)
        descs_data = [
            (ql.FdmSchemeDesc.CraigSneyd(), "Craig-Sneyd", True),
            (ql.FdmSchemeDesc.Hundsdorfer(), "Hundsdorfer", True),
            (ql.FdmSchemeDesc.ModifiedHundsdorfer(), "Mod. Hundsdorfer", True),
            (ql.FdmSchemeDesc.Douglas(), "Douglas", True),
            (ql.FdmSchemeDesc.CrankNicolson(), "Crank-Nicolson", True),
            (ql.FdmSchemeDesc.ImplicitEuler(), "Implicit", False),
            (ql.FdmSchemeDesc.TrBDF2(), "TR-BDF2", False)
        ]

        for scheme_enum, name, expected_spurious in descs_data:
            # FdmHestonSolver(processHandle, solverDesc, schemeDesc, leverageFctHandle (optional), mixingFactor (optional))
            solver = ql.FdmHestonSolver(process_h, solver_desc, scheme_enum)

            gammas = []
            spot_range = [99.0 + i * 0.1 for i in range(21)] # 99.0 to 101.0 inclusive (21 points)
            for x_spot in spot_range:
                gammas.append(solver.gammaAt(x_spot, v0))

            maximum_diff = 0.0
            if len(gammas) > 1: #float('-inf') if not gammas else gammas[0]
                 for i in range(1, len(gammas)):
                    diff = abs(gammas[i] - gammas[i-1])
                    if diff > maximum_diff:
                        maximum_diff = diff

            tol_oscillation = 0.01
            hasSpuriousOscillations = maximum_diff > tol_oscillation

            self.assertEqual(hasSpuriousOscillations, expected_spurious,
                             msg=(f"Spurious oscillation behavior mismatch for scheme: {name}\n"
                                  f"    Oscillations observed: {hasSpuriousOscillations} (max_diff={maximum_diff:.4f})\n"
                                  f"    Oscillations expected: {expected_spurious}"))

    def testAmericanCallPutParity(self):
        print("Testing call/put parity for American options under the Heston model...")
        dc = ql.Actual365Fixed()
        today = ql.Date(15, ql.April, 2022)
        ql.Settings.instance().evaluationDate = today

        # OptionSpec struct from C++
        # spot, strike, maturityInDays, r, q, v0, kappa, theta, sig, rho
        option_specs = [
            {'spot': 100.0, 'strike': 90.0, 'maturity_days': 365,
             'r': 0.02, 'q': 0.15, 'v0': 0.25, 'kappa': 1.0,
             'theta': 0.09, 'sig': 0.5, 'rho': -0.75},
            {'spot': 100.0, 'strike': 90.0, 'maturity_days': 365,
             'r': 0.05, 'q': 0.20, 'v0': 0.5, 'kappa': 1.0,
             'theta': 0.05, 'sig': 0.75, 'rho': -0.9}
        ]

        xGrid = 200; vGrid = 25; timeStepsPerYear = 50

        for i, spec in enumerate(option_specs):
            maturityDate = today + ql.Period(spec['maturity_days'], ql.Days)
            maturityTime = dc.yearFraction(today, maturityDate)
            tGrid = int(maturityTime * timeStepsPerYear)

            exercise = ql.AmericanExercise(today, maturityDate)

            # Call Option
            call_process = ql.HestonProcess(
                ql.YieldTermStructureHandle(ql.FlatForward(today, spec['r'], dc)),
                ql.YieldTermStructureHandle(ql.FlatForward(today, spec['q'], dc)),
                ql.QuoteHandle(ql.SimpleQuote(spec['spot'])),
                spec['v0'], spec['kappa'], spec['theta'], spec['sig'], spec['rho']
            )
            call_model = ql.HestonModel(call_process)
            callOption = ql.VanillaOption(
                ql.PlainVanillaPayoff(ql.Option.Call, spec['strike']), exercise)
            callOption.setPricingEngine(
                ql.FdHestonVanillaEngine(call_model, tGrid, xGrid, vGrid))
            callNpv = callOption.NPV()

            # Put Option (transformed parameters)
            put_spot = spec['strike']
            put_strike = spec['spot']
            put_r = spec['q']
            put_q = spec['r']
            put_v0 = spec['v0']

            # kappa_star = kappa - sigma_v * rho
            put_kappa_star = spec['kappa'] - spec['sig'] * spec['rho']
            # theta_star = (kappa * theta) / kappa_star
            if abs(put_kappa_star) < 1e-9: # Avoid division by zero
                 self.fail(f"Case {i}: put_kappa_star is zero, cannot compute put_theta_star.")
                 put_theta_star = 0 # Or some other handling
            else:
                put_theta_star = (spec['kappa'] * spec['theta']) / put_kappa_star

            put_sig = spec['sig']
            put_rho_star = -spec['rho']

            put_process = ql.HestonProcess(
                ql.YieldTermStructureHandle(ql.FlatForward(today, put_r, dc)),
                ql.YieldTermStructureHandle(ql.FlatForward(today, put_q, dc)),
                ql.QuoteHandle(ql.SimpleQuote(put_spot)),
                put_v0, put_kappa_star, put_theta_star, put_sig, put_rho_star
            )
            put_model = ql.HestonModel(put_process)
            putOption = ql.VanillaOption(
                ql.PlainVanillaPayoff(ql.Option.Put, put_strike), exercise)
            putOption.setPricingEngine(
                ql.FdHestonVanillaEngine(put_model, tGrid, xGrid, vGrid))
            putNpv = putOption.NPV()

            diff = abs(putNpv - callNpv)
            tol = 0.025
            self.assertLessEqual(diff, tol,
                                 msg=(f"American call/put parity failed for case {i}\n"
                                      f"    Put NPV   : {putNpv:.5f}\n"
                                      f"    Call NPV  : {callNpv:.5f}\n"
                                      f"    difference: {diff:.3e}"))

if __name__ == '__main__':
    print("C++ test suite: FdHestonTests")
    print("Python QuantLib version: " + ql.__version__)
    print("-------------------------------------------------------")
    # unittest.main(argv=['first-arg-is-ignored'], exit=False) # For running in environments like Jupyter

    suite = unittest.TestSuite()
    # Add tests selectively if needed, e.g., for debugging or if some are too slow
    suite.addTest(unittest.makeSuite(FdHestonTests))
    # suite.addTest(FdHestonTests('testFdmHestonVarianceMesher'))
    # suite.addTest(FdHestonTests('testFdmHestonBarrierVsBlackScholes')) # This one is long
    # suite.addTest(FdHestonTests('testFdmHestonBarrier'))
    # suite.addTest(FdHestonTests('testFdmHestonAmerican'))
    # suite.addTest(FdHestonTests('testFdmHestonIkonenToivanen'))
    # suite.addTest(FdHestonTests('testFdmHestonBlackScholes'))
    # suite.addTest(FdHestonTests('testFdmHestonEuropeanWithDividends'))
    # suite.addTest(FdHestonTests('testFdmHestonConvergence'))
    # suite.addTest(FdHestonTests('testFdmHestonIntradayPricing'))
    # suite.addTest(FdHestonTests('testMethodOfLinesAndCN'))
    # suite.addTest(FdHestonTests('testSpuriousOscillations'))
    # suite.addTest(FdHestonTests('testAmericanCallPutParity'))

    result = unittest.TextTestRunner(verbosity=2).run(suite)
    if not result.wasSuccessful():
        # This helps CI systems detect failure if the script doesn't exit with non-zero
        import sys
        sys.exit(1)

Important Notes:
testFdmHestonBarrierVsBlackScholes Data: The barrier_option_data list in testFdmHestonBarrierVsBlackScholes is extremely long (63 entries). I've put a placeholder and a comment. You'll need to copy the full data from the C++ code into that list of dictionaries for the test to be complete. I've included the C++ style tuple list and conversion code to make it easier to paste.
testFdmHestonIntradayPricing Skip Condition: The @unittest.skipIf condition for testFdmHestonIntradayPricing is a basic attempt to check if high-resolution dates are supported. It might need refinement based on specific QuantLib Python versions or configurations. If ql.Date(1,1,1901,1,1,1) is the same as ql.Date(1,1,1901), it implies time components are ignored.
setUp and tearDown: Added basic setUp and tearDown to save and restore the global evaluation date, as tests might modify it.
Relinkable Handles: In testFdmHestonIntradayPricing, ensure flatTermStructure_h.linkTo(...) and flatDividendTS_h.linkTo(...) are correctly used to update the term structures within the loop. The process will pick up changes from these handles.
American Exercise: The ql.AmericanExercise constructor in Python typically takes (earliestDate, latestDate). In many cases, earliestDate is the evaluation date.
Default Engine Parameters: I've tried to match the C++ default parameters for engines (like vGrid=50 if not specified) or use the explicitly provided ones from the C++ test constructors (e.g., FdHestonVanillaEngine(model, 100, 400) implies tGrid=100, xGrid=400, with other params default).
Error Messages: Added more descriptive error messages to assertAlmostEqual and other assertions to help pinpoint failures.
Running the Tests: The if __name__ == '__main__': block is set up to run all tests.
Tolerance for testFdmHestonVarianceMesher (slvMesher): The C++ test uses a specific tolerance of 0.001 for the last assertion in testFdmHestonVarianceMesher, which I've replicated.